In [19]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
from scipy.special import expit
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn import tree

In [20]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [21]:



API_KEY = "8edf5a3557214b83a8d24412250109"
local = "Curitiba"  # pode ser "Curitiba, Brazil" ou latitude,longitude
url = "http://api.weatherapi.com/v1/forecast.json"
params = {
    "key": API_KEY,
    "q": local,
    "lang": "pt",     # opcional: traduz descrições para português
    "aqi": "no"       # opcional: inclui dados de qualidade do ar
}

try:
    response = requests.get(url, params=params)
    response.raise_for_status()
    dados = response.json()
    
    # Exemplo de como acessar alguns dados:
    # temperatura = dados["current"]["temp_c"]
    # descricao = dados["current"]["condition"]["text"]
    # umidade = dados["current"]["humidity"]
    # vento = dados["current"]["wind_kph"]

    # print(f"Tempo em {dados['location']['name']}: {descricao.capitalize()}")
    # print(f"Temperatura: {temperatura} °C")
    # print(f"Umidade: {umidade}%")
    # print(f"Vento: {vento} km/h")
   # print(dados)

except requests.exceptions.HTTPError as err:
    print(f"Erro HTTP: {err} — código: {response.status_code}")
except ValueError:
    print("Erro ao decodificar JSON.")
except Exception as e:
    print(f"Ocorreu um erro: {e}")

df = pd.json_normalize(dados)
df

    


,location.name,location.region,location.country,location.lat,location.lon,location.tz_id,location.localtime_epoch,location.localtime,current.last_updated_epoch,current.last_updated,current.temp_c,current.temp_f,current.is_day,current.condition.text,current.condition.icon,current.condition.code,current.wind_mph,current.wind_kph,current.wind_degree,current.wind_dir,current.pressure_mb,current.pressure_in,current.precip_mm,current.precip_in,current.humidity,current.cloud,current.feelslike_c,current.feelslike_f,current.windchill_c,current.windchill_f,current.heatindex_c,current.heatindex_f,current.dewpoint_c,current.dewpoint_f,current.vis_km,current.vis_miles,current.uv,current.gust_mph,current.gust_kph,current.short_rad,current.diff_rad,current.dni,current.gti,forecast.forecastday
0,Curitiba,Parana,Brazil,-25.4167,-49.25,America/Sao_Paulo,1757644541,2025-09-11 23:35,1757644200,2025-09-11 23:30,15.1,59.2,0,Parcialmente nublado,//cdn.weatherapi.com/weather/64x64/night/116.png,1003,6.5,10.4,104,ESE,1023.0,30.21,0.0,0.0,72,0,15.1,59.2,12.4,54.3,13.2,55.7,11.1,52.0,10.0,6.0,0.0,9.9,15.9,0,0,0,0,"[{'date': '2025-09-11', 'date_epoch': 17575488..."


In [22]:
forecast_list = dados["forecast"]["forecastday"]
df_forecast = pd.json_normalize(forecast_list)
df_forecast


,date,date_epoch,hour,day.maxtemp_c,day.maxtemp_f,day.mintemp_c,day.mintemp_f,day.avgtemp_c,day.avgtemp_f,day.maxwind_mph,day.maxwind_kph,day.totalprecip_mm,day.totalprecip_in,day.totalsnow_cm,day.avgvis_km,day.avgvis_miles,day.avghumidity,day.daily_will_it_rain,day.daily_chance_of_rain,day.daily_will_it_snow,day.daily_chance_of_snow,day.condition.text,day.condition.icon,day.condition.code,day.uv,astro.sunrise,astro.sunset,astro.moonrise,astro.moonset,astro.moon_phase,astro.moon_illumination,astro.is_moon_up,astro.is_sun_up
0,2025-09-11,1757548800,"[{'time_epoch': 1757559600, 'time': '2025-09-1...",27.9,82.2,10.2,50.4,18.2,64.8,7.8,12.6,0.0,0.0,0.0,10.0,6.0,56,0,0,0,0,Sol,//cdn.weatherapi.com/weather/64x64/day/113.png,1000,2.1,06:18 AM,06:09 PM,10:32 PM,08:39 AM,Waning Gibbous,86,1,0


In [23]:
hour_list = dados["forecast"]["forecastday"][0]["hour"]
df_hour = pd.json_normalize(hour_list)
df_hour


,time_epoch,time,temp_c,temp_f,is_day,wind_mph,wind_kph,wind_degree,wind_dir,pressure_mb,pressure_in,precip_mm,precip_in,snow_cm,humidity,cloud,feelslike_c,feelslike_f,windchill_c,windchill_f,heatindex_c,heatindex_f,dewpoint_c,dewpoint_f,will_it_rain,chance_of_rain,will_it_snow,chance_of_snow,vis_km,vis_miles,gust_mph,gust_kph,uv,short_rad,diff_rad,dni,gti,condition.text,condition.icon,condition.code
0,1757559600,2025-09-11 00:00,11.2,52.1,0,3.4,5.4,75,ENE,1022.0,30.19,0.0,0.0,0.0,88,13,11.0,51.8,11.0,51.8,11.2,52.1,9.3,48.7,0,0,0,0,10.0,6.0,7.0,11.3,0.0,0.00,0.00,0.00,0.00,Céu limpo,//cdn.weatherapi.com/weather/64x64/night/113.png,1000
1,1757563200,2025-09-11 01:00,10.8,51.4,0,2.7,4.3,93,E,1022.0,30.18,0.0,0.0,0.0,89,12,10.9,51.6,10.9,51.6,10.8,51.4,9.0,48.2,0,0,0,0,10.0,6.0,5.6,9.1,0.0,0.00,0.00,0.00,0.00,Céu limpo,//cdn.weatherapi.com/weather/64x64/night/113.png,1000
2,1757566800,2025-09-11 02:00,10.6,51.0,0,2.5,4.0,54,NE,1021.0,30.16,0.0,0.0,0.0,88,9,10.7,51.3,10.7,51.3,10.6,51.0,8.7,47.7,0,0,0,0,10.0,6.0,5.2,8.3,0.0,0.00,0.00,0.00,0.00,Céu limpo,//cdn.weatherapi.com/weather/64x64/night/113.png,1000
3,1757570400,2025-09-11 03:00,10.5,50.8,0,2.9,4.7,13,NNE,1021.0,30.14,0.0,0.0,0.0,85,0,10.4,50.7,10.4,50.7,10.5,50.8,8.0,46.4,0,0,0,0,10.0,6.0,6.1,9.8,0.0,0.00,0.00,0.00,0.00,Céu limpo,//cdn.weatherapi.com/weather/64x64/night/113.png,1000
4,1757574000,2025-09-11 04:00,10.5,50.8,0,2.2,3.6,342,NNW,1021.0,30.15,0.0,0.0,0.0,81,0,10.8,51.4,10.8,51.4,10.5,50.8,7.4,45.4,0,0,0,0,10.0,6.0,4.7,7.6,0.0,0.00,0.00,0.00,0.00,Céu limpo,//cdn.weatherapi.com/weather/64x64/night/113.png,1000
5,1757577600,2025-09-11 05:00,10.2,50.4,0,2.2,3.6,316,NW,1021.0,30.16,0.0,0.0,0.0,83,0,10.5,50.9,10.5,50.9,10.2,50.4,7.5,45.5,0,0,0,0,10.0,6.0,4.7,7.6,0.0,0.00,0.00,0.00,0.00,Céu limpo,//cdn.weatherapi.com/weather/64x64/night/113.png,1000
6,1757581200,2025-09-11 06:00,11.1,52.0,0,2.5,4.0,317,NW,1022.0,30.18,0.0,0.0,0.0,80,0,11.4,52.4,11.4,52.4,11.1,52.0,7.8,46.0,0,0,0,0,10.0,6.0,5.2,8.3,0.0,0.00,0.00,0.00,0.00,Céu limpo,//cdn.weatherapi.com/weather/64x64/night/113.png,1000
7,1757584800,2025-09-11 07:00,15.0,58.9,1,3.1,5.0,307,NW,1022.0,30.17,0.0,0.0,0.0,63,0,15.4,59.7,15.4,59.7,15.0,58.9,8.0,46.5,0,0,0,0,10.0,6.0,4.8,7.7,0.2,7.12,2.62,6.25,5.48,Sol,//cdn.weatherapi.com/weather/64x64/day/113.png,1000
8,1757588400,2025-09-11 08:00,18.8,65.9,1,4.0,6.5,304,NW,1021.0,30.16,0.0,0.0,0.0,46,0,18.8,65.9,18.8,65.9,18.8,65.9,6.9,44.4,0,0,0,0,10.0,6.0,4.6,7.5,1.2,51.94,11.38,49.35,37.91,Sol,//cdn.weatherapi.com/weather/64x64/day/113.png,1000
9,1757592000,2025-09-11 09:00,22.1,71.9,1,4.9,7.9,293,WNW,1021.0,30.14,0.0,0.0,0.0,33,0,23.6,74.4,22.2,71.9,23.6,74.4,5.0,40.9,0,0,0,0,10.0,6.0,5.7,9.1,3.1,122.05,19.98,117.80,87.32,Sol,//cdn.weatherapi.com/weather/64x64/day/113.png,1000


In [24]:
# Lista apenas com as colunas importantes
colunas_importantes = [
    "time",
    "feelslike_c",
    "humidity",
    "cloud",
    "uv",
    "wind_kph",
    "will_it_rain",
    "chance_of_rain",
]

df_limpo = df_hour[colunas_importantes]
df_limpo.sample(5)

,time,feelslike_c,humidity,cloud,uv,wind_kph,will_it_rain,chance_of_rain
0,2025-09-11 00:00,11.0,88,13,0.0,5.4,0,0
5,2025-09-11 05:00,10.5,83,0,0.0,3.6,0,0
11,2025-09-11 11:00,24.8,19,0,8.0,9.4,0,0
1,2025-09-11 01:00,10.9,89,12,0.0,4.3,0,0
19,2025-09-11 19:00,18.4,54,10,0.0,5.8,0,0


In [ ]:
# Cria uma tabela associando estilos de roupa com dados do clima
df_tabela = pd.DataFrame({
    "estilo de roupa": ["casual", "esportivo", "social", "streetwear"],
    "data e hora": df_limpo["time"].values[:4],
    "temperatura": df_limpo["feelslike_c"].values[:4],
    "chance de chuva": df_limpo["chance_of_rain"].values[:4],
    "umidade": df_limpo["humidity"].values[:4],
    "uv": df_limpo["uv"].values[:4],
})

#peças de roupas por estilo especifico 

roupas_por_estilo = {
"casual para calor": ["camiseta leve", "shorts jeans"],
"casual para frio": [],


}


df_tabela


,estilo de roupa,data e hora,temperatura,chance de chuva,umidade,uv
0,casual,2025-09-11 00:00,11.0,0,88,0.0
1,esportivo,2025-09-11 01:00,10.9,0,89,0.0
2,social,2025-09-11 02:00,10.7,0,88,0.0
3,streetwear,2025-09-11 03:00,10.4,0,85,0.0
